# STR detection

The STR profile is generated using two approaches; one based on the length of the STR regions the other on the  actual sequence.

Part 1) Splicing the reads into subreads

Part 2A) Mapping the sequence data against a reference

Part 2B) Generating a length based profile



Part Christophe moet nog een bekeken worden. 
Zeker het plotten en dergelijke

-Obv 3 fouten in primers een locus identificeren. 
Dit deel is achteraan terug te vinden. 


## PART 1: Splicing read into subreads. 

-Loading Primers

-Loading Sequencing data

-Using fuzzy Regex to identify the primers and slice the reads.

### Loading primers and sequencing data

In [38]:
# Load primer sequences (comma separated: str_name,forward_primer_seq,reverse_primer_seq)


primerFile = '/home/senne/nanopore/Multiplex/Reference_sequences/STR/primersequences.csv'
#Analoog aan SNP primers
primerData = {}

with open(primerFile) as f:
  for l in f:
    l = l.strip()
    
    # Ignore the column header line (should start with a '#')
    if l.startswith('#'):
      continue
    
    locus, fPrimer, rPrimer = l.split(',')
    primerData[locus] = {'f': fPrimer, 'r': rPrimer}
    
print('Loaded primers for {} loci'.format(len(primerData)))

Loaded primers for 14 loci


In [39]:
# Load sequencing data (ligated loci)
#

readFile = '/home/senne/nanopore/Multiplex/results/barcode_NB01.fasta'
#/home/senne/nanopore/Multiplex/results/barcode_{}.fasta
readData = {}

with open(readFile) as f:
  cnt = 0

  for l in f:
    cnt += 1
    if cnt % 2 == 1:
      seqName = l.strip()
    elif cnt % 2 == 0:
      seqData = l.strip()
      readData[seqName] = seqData
    else:
      continue
            
print('Loaded {} sequences'.format(len(readData)))

Loaded 9851 sequences


### Using a fuzzy regex to splice the reads into subreads

In [40]:
# Utility functions
#

def reverseComplement(seq):
  transTab = str.maketrans('agctyrwskmdvhbAGCTYRWSKMDVHB', 'tcgarywsmkhbdvTCGARYWSMKHBDV')
  return seq.translate(transTab)[::-1]

In [41]:
import regex

maxMisMatch  = 3
matchPosData = {}
ampliconSeq  = []
ampLenData   = {}

for locus in sorted(primerData):
    # DEBUG
    #if len(ampLenData)==4: break

    # Init
    ampLenData[locus] = {}
        
    # Regex
    pFor   = regex.compile('(?e)({}){{e<={}}}'.format(primerData[locus]['f'], maxMisMatch))
    pForRc = regex.compile('(?e)({}){{e<={}}}'.format(reverseComplement(primerData[locus]['f']), maxMisMatch))
    pRev   = regex.compile('(?e)({}){{e<={}}}'.format(primerData[locus]['r'], maxMisMatch))
    pRevRc = regex.compile('(?e)({}){{e<={}}}'.format(reverseComplement(primerData[locus]['r']), maxMisMatch))
    
    allowedPairs      = [(pFor, pRevRc), (pRev, pForRc)]
    allowedPairSense  = ('sense', 'anti-sense')
    allowedPairSeqLen = 90
    minAmpliconSize   = 50
    maxAmpliconSize   = 500
    readCnt           = 0
    
    # Print locus name to indicate progress
    print(locus)
    
    for seq in readData:
        readCnt += 1
        
        # DEBUG
        #if readCount == 100: break
        
        #for pair in allowedPairs:
        for i in range(len(allowedPairs)):
            pair    = allowedPairs[i]
            sense   = allowedPairSense[i]
            posList = [[],[]]
            
            for match in pair[0].finditer(readData[seq]):
                b,e = match.span()
                posList[0].append(b) # Including the primer
                #posList[0].append(e+1) # Excluding the primer, need to refine this if the error is the last nucleotide
                
            for match in pair[1].finditer(readData[seq]):
                b,e = match.span()
                posList[1].append(e) # Including the primer
                #posList[1].append(b-1) # Excluding the primer, need to refine this if the error is the first nucleotide
                
            # Retain possible pairs based on amplicon size
            if len(posList[0]) and len(posList[1]):
                # DEBUG
                #print('Found both {} primers in read {}: {}, {}'.format(locus, readCnt, posList[0], posList[1]))
                
                for p1 in posList[0]:
                    for p2 in posList[1]:
                        if minAmpliconSize < (p2 - p1) < maxAmpliconSize:
                            # We have a potential amplicon?
                            #ampLen  = p2 - p1
                            sense   = '{} - {} ({})'.format(p1, p2, sense)
                            ampSeq  = readData[seq][p1:p2]
                            ampLen  = len(ampSeq)
                            ampName = '{} {} ({}): {} {}'.format(len(ampliconSeq) + 1, locus, ampLen, sense, seq)
                            ampliconSeq.append((ampName, ampSeq))
                            
                            # Keep amplicon lengths
                            if ampLen in ampLenData[locus]:
                                ampLenData[locus][ampLen] += 1
                            else:
                                ampLenData[locus][ampLen] = 1
                            
                            # DEBUG
                            #print('{}, read:{}, amplen={}, {}'.format(locus, readCnt, ampLen, sense))
                                                    
    
# Save potential amplicons
outFile = '/home/senne/nanopore/Multiplex/results/STR/STR_BC1-{}mism.fasta'.format(maxMisMatch)
with open(outFile, 'w') as f:
    for amplicon in ampliconSeq:
        f.write('>' + amplicon[0] + '\n')
        f.write(amplicon[1] + '\n')
    
print('Done')

Amelogenine
D13S317
D16S539
D18S51
D21S11
D3S1358
D5S818
D7S820
D8S1179
FGA
SE33
TH01
TPOX
vWA
Done


In [42]:
%%bash
cd /home/senne/nanopore/Multiplex/results/STR/
grep -c '>' STR_BC8-3mism.fasta

758


##PART 2A: Mapping against database containing all known alleles

-Mapping using BWA and SAM

-Extracting the uniquely mapped and once with a mapping score =0 form sam file

-Writing to a txt file ready for excel

### Input

In [7]:
# Init
#
NBarcode = 1

strFile   = '/home/senne/nanopore/Multiplex/Reference_sequences/STR/STR_database.fasta' # REMOVE FIRST LINE IN ORIGINAL FILE (BREAKS FASTA FORMAT)
#Dit moet dus de databank zijn van alle bestaande STR allelen voor alle loci. 
readFile  = '/home/senne/nanopore/Multiplex/results/STR/STR_BC{}-3mism.fasta'.format(NBarcode)
#resultDir = '/home/senne/nanopore/Multiplex/results/STR/'
fastq_file_name= '/home/senne/nanopore/Multiplex/results/STR/STR_BC{}-3mism.fasta'.format(NBarcode)

samfile = '/home/senne/nanopore/Multiplex/results/STR/STR_BCont2d{}.sam'.format(NBarcode)
bamfile = '/home/senne/nanopore/Multiplex/results/STR/STR_BC{}ont2d.bam'.format(NBarcode)
bambaifile = '/home/senne/nanopore/Multiplex/results/STR/STR_BC{}ont2d.bam.bai'.format(NBarcode)
txtfile = '/home/senne/nanopore/Multiplex/results/STR/STR_BC{}ont2d.txt'.format(NBarcode)

bwa       = '/opt/tools/bwa-0.7.15'            # v0.7.5
samtools  = '/opt/tools/samtools-1.3.1' # v1.3.1
bcftools  = '/opt/tools/bcftools-1.3.1' # v1.3.1


## oppassen voor de bwa versie!! ##
# Check
!ls {strFile}
print('Number of STRs in reference file:')
!grep -c ">" {strFile}

/home/senne/nanopore/Multiplex/Reference_sequences/STR/STR_database.fasta
Number of STRs in reference file:
141


In [8]:

#SNP and STR data are present in this fasta file
hist_array = []
hist_arrayG = []
hist_arrayA = []
hist_arrayC = []
hist_arrayT = []
with open(fastq_file_name, "rb") as infile:
    for line in infile:
        if line.startswith(b'G'):
            read_length = len(line[:-1]) #Last char is \n
            #histogram_data[read_length] += 1
            hist_arrayG.append(read_length)
with open(fastq_file_name, "rb") as infile:
    for line in infile:
        if line.startswith(b'A'):
            read_length = len(line[:-1]) #Last char is \n
            #histogram_data[read_length] += 1
            hist_arrayA.append(read_length) 
with open(fastq_file_name, "rb") as infile:
    for line in infile:
        if line.startswith(b'C'):
            read_length = len(line[:-1]) #Last char is \n
            #histogram_data[read_length] += 1
            hist_arrayC.append(read_length) 
with open(fastq_file_name, "rb") as infile:
    for line in infile:
        if line.startswith(b'T'):
            read_length = len(line[:-1]) #Last char is \n
            #histogram_data[read_length] += 1
            hist_arrayT.append(read_length)

hist_array.extend(hist_arrayA)
hist_array.extend(hist_arrayC)
hist_array.extend(hist_arrayG)
hist_array.extend(hist_arrayT)

import numpy as np

len (hist_array), np.mean(hist_array), np.median(hist_array) , max(hist_array), np.std(hist_array)

(5541, 188.4722974192384, 168.0, 499, 68.680266923821009)

### Mapping using BWA and SAM

In [9]:
# Map reads to reference sequences
#

# Build index of the references
!{bwa} index {strFile}

# Map reads
!{bwa} mem -x ont2d {strFile} {readFile} > {samfile}

print('done')

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.15-r1140
[main] CMD: /opt/tools/bwa-0.7.15 index /home/senne/nanopore/Multiplex/Reference_sequences/STR/STR_database.fasta
[main] Real time: 0.011 sec; CPU: 0.015 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 5541 sequences (1044325 bp)...
[M::mem_process_seqs] Processed 5541 reads in 5.398 CPU sec, 5.396 real sec
[main] Version: 0.7.15-r1140
[main] CMD: /opt/tools/bwa-0.7.15 mem -x ont2d /home/senne/nanopore/Multiplex/Reference_sequences/STR/STR_database.fasta /home/senne/nanopore/Multiplex/results/STR/STR_BC1-3mism.fasta
[main] Real time: 5.419 sec; CPU: 5.415 sec
done


In [10]:
# Make sorted bam and index
#
!{samtools} view -Sbu {samfile} | {samtools} sort -o {bamfile} -
!{samtools} index {bamfile} {bambaifile}
print('Done')

Done


### Extracting the uniquely mapped and once with a mapping score = 0 form sam file

Yannick hier nog eens naar laten kijken.

In [12]:
%%bash 
cd /home/senne/nanopore/Multiplex/results/STR/
echo -n "" > STR_BC1ont2d.txt
for i in "Amelogenine_X" "Amelogenine_Y" "D3S1358_13" "D3S1358_14" "D3S1358_15" "D3S1358_16" "D3S1358_17" "D3S1358_18" "D3S1358_19" "D21S11_27" "D21S11_28"  "D21S11_29" "D21S11_30" "D21S11_31" "D21S11_31.2" "D21S11_32" "D21S11_32.2" "D21S11_33.2" "TH01_5" "TH01_6" "TH01_7" "TH01_8" "TH01_9" "TH01_9.3" "TH01_10" "TH01_11" "D16S539_9" "D16S539_10" "D16S539_11" "D16S539_12" "D16S539_13" "D16S539_14" "D16S539_15" "D7S820_7" "D7S820_8" "D7S820_9" "D7S820_10" "D7S820_11" "D7S820_12" "D7S820_13" "D7S820_14" "D13S317_8" "D13S317_9" "D13S317_10" "D13S317_11" "D13S317_12" "D13S317_13" "D13S317_14" "D13S818_15" "D13S818_16" "D18S51_12" "D18S51_13" "D18S51_14" "D18S51_15" "D18S51_16" "D18S51_17" "D18S51_18" "D18S51_19" "D18S51_20" "D5S818_9" "D5S818_10" "D5S818_11" "D5S818_12" "D5S818_13" "D5S818_14" "D5S818_15" "D5S818_16" "D5S818_17" "D5S818_18" "D8S1179_8" "D8S1179_9" "D8S1179_10" "D8S1179_12" "D8S1179_13" "D8S1179_14" "D8S1179_15" "D8S1179_16" "D8S1179_17" "D8S1179_18" "FGA_18" "FGA_19" "FGA_20" "FGA_21" "FGA_22" "FGA_23" "FGA_24" "FGA_25" "FGA_26" "FGA_27" "FGA_28" "TPOX_6" "TPOX_7" "TPOX_8" "TPOX_9" "TPOX_10" "TPOX_11" "TPOX_12" "TPOX_13" "vWA_15" "vWA_16" "vWA_17" "vWA_18" "vWA_19" "vWA_20" "SE33_15" "SE33_16" "SE33_18" "SE33_19" "SE33_20" "SE33_21" "SE33_21.2"  "SE33_22.2" "SE33_23.2" "SE33_24.2" "SE33_25.2" "SE33_26.2" "SE33_27.2" "SE33_28.2" "SE33_29.2" "SE33_30.2" "SE33_31.2"; do 
    echo -n "$i;" >> STR_BC1ont2d.txt
    awk '$2 < 1' STR_BCont2d1.sam | awk '$5 > 0'| cut -f3 -d$'\t'| grep -c $i >> STR_BC1.txt
done


###Klaar maken voor excel

In [12]:
%%bash
cd /home/senne/nanopore/Multiplex/results/test/

tr "_" "\t" < STR_BC1.txt | tr ";" "\t" > Mapped_profile_test.txt



##Amelogenine

##PART 2B: Determining the length of the sequenced subreads

Plot Christophe zorgt voor een fasta met daarin de subreads
Hoe krijgen we de reeds gespliced reads in deze pipeline? 

Daarnaast wordt er nog een histogram gemaakt en gaussiaanse curve opgeplot.



In [4]:
# Analyzing full reads
# Reflections
#      - Currently generic peak detection and fitting of gaussian curve
#      - Future: if enough data, machine learning should be used
#      - scikit-learn
#      - Literature
#      - http://stackoverflow.com/questions/10143905/python-two-curve-gaussian-fitting-with-non-linear-least-squares
#      - http://stackoverflow.com/questions/19206332/gaussian-fit-for-python
#      - http://bioinformatics.oxfordjournals.org/content/22/17/2059.long
#      - http://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks_cwt.html
%matplotlib inline

from itertools import count
from math import floor,ceil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
sys.path.append('/home/senne/nanopore')
from MyFLq import complement, calculateAlleleNumber, Locus

loci = Locus.makeLocusDict(('csv', '/home/senne/nanopore/SNP/known_SNP_sequence/SNP_loci_config2.csv'))
exit()
#Functions
def bpTOfloat(bpLength,locusType):
    return int(bpLength)+(int(str(bpLength).split('.')[1]) if '.' in str(bpLength) else 0)/locusType

def floatTObp(floatLength,locusType):
    return int(floatLength)+(int(str(floatLength).split('.')[1]) if '.' in str(floatLength) else 0)*locusType

def calculateOriginalLength(repeats,locusType,refsize,refrepeats):
    fullRefRepeats,partialRefRepeat = str(float(refrepeats)).split('.')
    fullRepeats,partialRepeat = str(float(repeats)).split('.')
    return refsize+locusType*(int(fullRepeats)-int(fullRefRepeats))+(int(partialRepeat)-int(partialRefRepeat))

def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

def gaus2(x,a_1,x0_1,sigma_1,a_2,x0_2,sigma_2):
    return a_1*np.exp(-(x-x0_1)**2/(2*sigma_1**2))+a_2*np.exp(-(x-x0_2)**2/(2*sigma_2**2))

# Main class
class LigatedRead:
    """
    Class that represents a ligated read of subreads.
    Methods allow to extract the subreads for further processing.
    The initial read can contain '&', which are divisions between
    already known subreads.
    """
    def __init__(self, read, loci,minSize1read=70, maxSize1read=500, maxPrimerErrors=0):
        self.read = read
        for l in loci:
            # Prep loci for use with MyFLq.calculateAlleleNumber
            loci[l]['ref_length']       = len(loci[l]['ref_sequence'])
            loci[l]['ref_alleleNumber'] = loci[l]['ref_number']
            
        self.loci                = loci
        self.maxSize1read        = maxSize1read
        self.minSize1read        = minSize1read
        self.maxPrimerErrors     = maxPrimerErrors
        self.averagePrimerLength = sum(len(self.loci[l]['ref_forwardP'])+
                                       len(self.loci[l]['ref_reverseP'])
                                       for l in self.loci)/(2*len(self.loci))
        
    def processPrimers(self,withPrimerErrors=0):
        # Find previous readfragments
        self.markpositions('&')
        # Primer errors setup
        if withPrimerErrors:
            # Calculate kmer size
            kmerSize = int(self.averagePrimerLength/(1+withPrimerErrors))  # k is half primer length
            # Kmer count in expected sequences
            self.reference_kmers = {}
            for l in self.loci:
                for seq in (self.loci[l]['ref_sequence'],complement(self.loci[l]['ref_sequence'])):
                    for i in range(len(seq)+1-kmerSize):
                        try: self.reference_kmers[seq[i:i+kmerSize]]+=1
                        except KeyError: self.reference_kmers[seq[i:i+kmerSize]]=1
        # Find primers
        for locus in self.loci:
            for primertype in ('ref_forwardP','ref_reverseP','ref_forwardP_c','ref_reverseP_c'):
                primer = (self.loci[locus][primertype] if not primertype.endswith('_c')
                          else complement(self.loci[locus][primertype[:-2]]))
                if not withPrimerErrors:
                    self.markpositions(primer,locus,primertype)
                else:
                    primerKmers = {i:primer[i:i+kmerSize] for i in range(0,len(primer)+1-kmerSize,kmerSize)}
                    if len(primer)%kmerSize != 0: primerKmers[kmerSize] = primer[-kmerSize:]
                    
                    # (YG) DEBUG
                    print('{} {} {} has {} k-mers (k={})'.format(locus, primertype, primer, len(primerKmers), kmerSize))
                    
                    for o in primerKmers:
                        # (YG) kmer must occur only once in reference sequence (which includes the primer sequences)
                        if self.reference_kmers[primerKmers[o]] == 1:
                            self.markpositions(primerKmers[o],locus,primertype,offset=o)
        # Remove duplicates TODO (make set, then sorted for list)
        # Sort on position
        self.primerPositions.sort(key = lambda x: x[0])

    def markpositions(self,pattern,locus=None,primertype=None,offset=0):
        try:
            currentPosition = self.read.find(pattern)
            while currentPosition != -1:
                self.primerPositions.append((currentPosition-offset,locus,primertype))
                currentPosition = self.read.find(pattern,currentPosition+1)
            
        except AttributeError:
            self.primerPositions = []
            self.markpositions(pattern,locus,primertype)

    def extractReads(self,filterArtefacts=True):
        self.subreads = []         # [((pos, locus, primertype), (pos, locus, primertype)),]
        pp = self.primerPositions  # [(pos, locus, primertype), ]
        for i in range(len(pp)-1):
            if (self.minSize1read < (pp[i+1][0]-pp[i][0]) < self.maxSize1read and   # 70 < ampliconsize < 500
                pp[i][1] == pp[i+1][1] and pp[i][2] and pp[i+1][2] and              # must be same locus, must have primertype
                pp[i][2][:6] != pp[i+1][2][:6] and                                  # cannot be both ref_for or both ref_rev
                (pp[i][2].endswith('_c') ^ pp[i+1][2].endswith('_c'))):             # one must end with _c
                self.subreads.append((pp[i],pp[i+1]))
        if filterArtefacts:
            nonArtifacts   = {('ref_forwardP', 'ref_reverseP_c'),
                              ('ref_reverseP', 'ref_forwardP_c')}
            self.artifacts = [s for s in self.subreads if (s[0][2],s[1][2]) not in nonArtifacts]
            self.subreads  = [s for s in self.subreads if (s[0][2],s[1][2]) in nonArtifacts]
            
            # (YG) DEBUG
            #print('Found {} subreads, {} artifacts'.format(len(self.subreads), len(self.artifacts)))
            #for artifact in self.artifacts:
            #  print('Artifact: {}'.format(artifact))

    def sortsubreads(self):
        # Sort first on length
        self.subreads.sort(key=lambda x: (x[1][0]-x[0][0])+len(self.loci[x[1][1]][x[1][2].replace('_c','')]))
        # Then on locus
        self.subreads.sort(key=lambda x: x[0][1])

    def exportReads(self,filename,mode='wt',type='fasta',locus=None,alleleLength=None,maxReads=None):
        ci = count(1)
        with open(filename,mode) as outfile:
            countBlankInSeq = 0
            for r in self.subreads:
                # (YG) sequence reconstruction: from startpos left primer match to startpos-1 right primer match + sequence of the right primer
                seq = self.read[r[0][0]:r[1][0]+len(self.loci[r[1][1]][r[1][2].replace('_c','')])]
                if ' ' in seq:
                    countBlankInSeq+=1
                    continue
                if 'reverse' in r[0][2]:
                    seq = complement(seq)
                    orientation = 'reverse'
                else: orientation = 'forward'
                outfile.write('>{} {} ({}): {} - {} ({})\n'.format(next(ci),r[0][1],len(seq),r[0][0],r[1][0],orientation))
                outfile.write(seq+'\n')
            if countBlankInSeq: print(countBlankInSeq,"reads contained blanks and were not exported")

    def histLengths(self):
        self.sortsubreads()
        self.histLengthData = {}
        for subr in self.subreads:
            locus = subr[0][1]
            # (YG) sequence reconstruction: from startpos left primer match to startpos-1 right primer match + sequence of the right primer
            seq = self.read[subr[0][0]:subr[1][0]+len(self.loci[locus][subr[1][2].replace('_c','')])]
            #if self.loci[locus]['locusType']:
            #    length = float(calculateAlleleNumber(seq,self.loci[locus]))
            #else:
            length = len(seq)
            try:
                self.histLengthData[locus][length]+=1
            except KeyError:
                if locus not in self.histLengthData: self.histLengthData[locus] = {}
                self.histLengthData[locus][length]=1

    def peakDetection(self,peak_min_width=8,peak_max_width=12,
                      peak_max_hight_diff=0.2,
                      includeRefProfile=False):
        """
        Work in progress
        First draft: peak detection on STR repeat number lenghts,
        but should be performed on bp length.

        Concept:
        - first scipy.signal.find_peaks_cwt for initial peak detection
        - select two most prominent peaks, if they are separated minimal distance
          and have minimal height difference
        - on those 1 or 2 peaks calculate gaussian fit
        - use mean and calculated sigma to esitmate likely allele lengths
        """
        
        ################################Is dit deel dan alleen nodig voor de piek detectie? #######################"
        from scipy import signal
        from scipy.optimize import curve_fit
        import numpy as np
        
        self.profile  = {}
        plotDimension = np.sqrt(len(self.histLengthData))
        fig,axes      = plt.subplots(ceil(plotDimension),ceil(len(self.histLengthData)/plotDimension),sharex=True,sharey=True)
        fig.set_size_inches(2*11.692, 2*8.267)
        
        for l,ax in zip(sorted(self.histLengthData),
                        (ax for row in axes for ax in row)):
            locusType = self.loci[l]['locusType']
            x         = sorted(self.histLengthData[l])
            x_range   = list(range(0,x[-1]+1))
            y         = [0 if i not in x else self.histLengthData[l][i] for i in x_range]
            peaks     = signal.find_peaks_cwt(y,np.arange(peak_min_width,peak_max_width))
            peaks     = [(x_range[p],y[p]) for p in peaks]
            peaks.sort(key = lambda x: x[1],reverse = True)
            
            # Potential peaks
            if len(peaks) > 1 and peaks[0][1]*peak_max_hight_diff < peaks[1][1]:
                peaks = peaks[:2]
            else:
                peaks = [peaks[0]]
                
            # Calculate gaussian fit
            x = np.array(x)
            y = np.array([self.histLengthData[l][i] for i in x])
            ax.plot(x,y,'b+',label=l, marker='.')
            try:
                if len(peaks) == 1:
                    popt,pcov       = curve_fit(gaus,x,y,p0=[max(y),peaks[0][0],peak_min_width])
                    self.profile[l] = ((calculateAlleleNumber(' '*int(round(popt[1])),self.loci[l]) if locusType
                                        else round(popt[1]), popt[2]/(locusType if locusType else 1)),)
                
                    ax.plot(x,gaus(x,*popt),'ro:',label='fit1', marker='.')
                elif len(peaks) == 2:
                    popt,pcov = curve_fit(gaus2,x,y,p0=[max(y),peaks[0][0],peak_min_width,
                                                        max(y),peaks[1][0],peak_min_width])
                    self.profile[l] = ((calculateAlleleNumber(' '*int(round(popt[1])),self.loci[l]) if locusType
                                        else round(popt[1]), popt[2]/(locusType if locusType else 1)),
                                       (calculateAlleleNumber(' '*int(round(popt[4])),self.loci[l]) if locusType
                                        else round(popt[4]), popt[5]/(locusType if locusType else 1)))
                    ax.plot(x,gaus2(x,*popt),'ro:',label='fit2', marker='.')
                else: self.profile[l] = None
            except RuntimeError:
                self.profile[l] = None
                
            if includeRefProfile and locusType:
                # Mark the reference profile allele lengths with vertical green lines
                for ra in self.referenceProfile[l]:
                    position = calculateOriginalLength(ra, locusType,
                                                       self.loci[l]['ref_length'],
                                                       self.loci[l]['ref_number'])
                    ax.plot((position,position), ax.get_ylim(),'g-')
                    
            ax.legend()
        plt.show(block=False)

    def CPI(self,populationFile):
        """
        Calculates combined probability of inclusion, aka
        random match probability.
        populationFile should be 'csv' formatted, with each line:
            <locus name>,<allele size>,<allele frequence>
        (without angular brackets)
        """
        # Reprocess self.profile to ranges of alleles
        self.profileCI = {}
        self.CPI_value = 1
        self.populationData = pd.read_csv(populationFile)
        self.CPI_unusedLoci = set(self.profile) - set(self.populationData['#Locus name'])        
        
        for l in self.profile:
            if l in self.CPI_unusedLoci: continue
            locusAlleles = self.populationData[self.populationData[
                self.populationData.columns[0]]==l]
            locusAN = locusAlleles["Allele number"]
            # TODO float/allele number issue
            alleleRanges = [(float(g[0])-g[1],float(g[0])+g[1]) for g in self.profile[l]]
            if len(alleleRanges) == 2:
                alleleRanges.sort(key = lambda x: x[0])
                if alleleRanges[0][1] > alleleRanges[1][0]:
                    mean = (alleleRanges[0][1] + alleleRanges[1][0])/2
                    alleleRanges = [(alleleRanges[0][0],mean),(mean,alleleRanges[1][1])]
                    # TODO check if with 'mean' is best strategy
            self.profileCI[l] = alleleRanges
            af1 = locusAlleles[(locusAN >= alleleRanges[0][0]) &
                               (locusAN < alleleRanges[0][1])]["Allele Frequency"].sum()
            if len(alleleRanges) == 1:
                locusProbability = af1**2
            else:
                af2 = locusAlleles[(locusAN >= alleleRanges[1][0]) &
                                   (locusAN < alleleRanges[1][1])]["Allele Frequency"].sum()
                locusProbability = 2*af1*af2
            self.CPI_value*=locusProbability

    def linkReferenceProfile(self,referenceProfileFile):
        self.referenceProfile = {}
        with open(referenceProfileFile) as inprofile:
            for line in inprofile:
                if line.startswith('#'): continue
                line = line.strip().split(',')
                self.referenceProfile[line[0]] = (float(line[1]),float(line[2]))
                
# Processing reads
fastq    = open("/home/senne/nanopore/SNP/Nanopore_data/ligatedSNPs.fastq")
c        = count(0)
reads    = [line for line in fastq if next(c)%4 == 1]
allreads = '&'.join(reads)

#ligread = LigatedRead('&'.join(reads),loci)
ligread = LigatedRead(allreads,loci)
#ligread.linkReferenceProfile('/media/sf_vm_shared/nanopore/Profile9948A')
ligread.processPrimers(withPrimerErrors=1)
ligread.extractReads()
ligread.histLengths()
ligread.exportReads('/home/senne/nanopore/SNP/Nanopore_data/separatedLigatedSNPs_minlen.fasta')
ligread.peakDetection(includeRefProfile=True)
ligread.CPI(populationFile = '/home/senne/nanopore/SNP/Europe_SNP_freq.csv')
print("RPM value profile:", ligread.CPI_value)

# (YG) Display profile
print()
print('Estimated profile:')
for locus in sorted(ligread.profile):
    alleles = ':'.join([str(a[0]) for a in ligread.profile[locus]])
    print('  {} {}'.format(locus, alleles))

# Calculate an original length
#calculateOriginalLength(10.3,
                       # ligread.loci['D13S317']['locusType'],
                        #ligread.loci['D13S317']['ref_length'],
                        #ligread.loci['D13S317']['ref_number'])

All reference sequences should start and end with the primer.
 This is not the case for locus rs1490413 in your configuration.


SystemExit: 1

To exit: use 'exit', 'quit', or Ctrl-D.


Loaded primers for 52 loci


Loaded 21875 sequences


rs1005533
rs1015250
rs1024116
rs1028528
rs1029047
rs1031825
rs10495407
rs1335873
rs1355366
rs1357617
rs1360288
rs1382387
rs1413212
rs1454361
rs1463729
rs1490413
rs1493232
rs1528460
rs1886510
rs1979255
rs2016276
rs2040411
rs2046361
rs2056277
rs2076848
rs2107612
rs2111980
rs251934
rs2830795
rs2831700
rs354439
rs717302
rs719366
rs722098
rs727811
rs729172
rs733164
rs735155
rs737681
rs740910
rs763869
rs8037429
rs826472
rs873196
rs876724
rs891700
rs901398
rs907100
rs914165
rs917118
rs938283
rs964681
Done
